In [1]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.session import SparkSession
from splink import Splink
from splink.jar_fallback import jw_sim_py

In [2]:
print("PySpark Application Started")

spark = SparkSession \
    .builder \
    .appName("Splink") \
    .master("local[*]") \
    .getOrCreate()

print(spark.sparkContext.appName)

df = spark.read.option("header", "true").csv("/home/abhishekks446/data/data3/netflix_titles.csv")
print("the data is:")

# df.createOrReplaceTempView("table1")
# df_new=spark.sql("select * from table1 limit 8000")
df_new=df

PySpark Application Started


22/07/06 20:43:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/07/06 20:43:56 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Splink
the data is:


In [3]:
df_new.count()

9900

In [4]:

settings = {
    "link_type": "dedupe_only",
    "blocking_rules": [
        "l.type = r.type and l.director = r.director"
#         "l.director = r.director"
#         "l.first_name = r.first_name and l.city = r.city"
    ],
    "comparison_columns": [

        {
            "col_name": "title",
            "num_levels": 4,
            "term_frequency_adjustments": True
        },
        {
            "col_name": "director"
        }
#        
    ]
#     "additional_columns_to_retain": ["group"]
}

linker = Splink(settings,df_new, spark)
df_e = linker.get_scored_comparisons()

/home/abhishekks446/pyspark/my_project_dir/my_project_env/lib/python3.8/site-packages/splink/case_statements.py:20: UserWarning: 

Custom string comparison functions such as jaro_winkler_sim are not available in Spark
Or you did not pass 'spark' (the SparkSession) into 'Model' 
You can import these functions using the scala-udf-similarity-0.0.9.jar provided with Splink.
You will need to add it by correctly configuring your spark config
For example in Spark 2.4.5

from pyspark.sql import SparkSession, types
from pyspark.context import SparkConf, SparkContext
conf.set('spark.driver.extraClassPath', '/home/abhishekks446/pyspark/my_project_dir/my_project_env/lib/python3.8/site-packages/splink/jars/scala-udf-similarity-0.0.9.jar') # Not needed in spark 3
conf.set('spark.jars', '/home/abhishekks446/pyspark/my_project_dir/my_project_env/lib/python3.8/site-packages/splink/jars/scala-udf-similarity-0.0.9.jar')
spark.udf.registerJavaFunction('jaro_winkler_sim','uk.gov.moj.dash.linkage.JaroWinkle

In [5]:
df_new=df_e.toPandas().sort_values("match_probability", ascending=False)


In [6]:
dedupe=df_new.loc[df_new['match_probability'] > 0.95]

In [7]:
dedupe.count()


match_weight         742
match_probability    742
unique_id_l          742
unique_id_r          742
title_l              742
title_r              742
gamma_title          742
director_l           742
director_r           742
gamma_director       742
dtype: int64

In [ ]:
# df_1 = spark.read.option("header", "true").csv("/home/abhishekks446/data/netflix_titles.csv")
# df_1.count()

In [ ]:
# df_e.coalesce(1).write.format("csv").option("header","true").mode("overwrite").save("/home/abhishekks446/data/output")

In [8]:
dedupe.head(50)

,match_weight,match_probability,unique_id_l,unique_id_r,title_l,title_r,gamma_title,director_l,director_r,gamma_director
4352,10.145862,0.999118,s812,s9619,Super Monsters: Once Upon a Rhyme,Super Monsters: Once Upon a Rhyme,3,Steve Ball,Steve Ball,1
2321,10.145862,0.999118,s160,s8967,Love in a Puff,Love in a Puff,3,Pang Ho-cheung,Pang Ho-cheung,1
2304,10.145862,0.999118,s625,s9432,Prime Time,Prime Time,3,Jakub Piątek,Jakub Piątek,1
2308,10.145862,0.999118,s456,s9263,Emicida: AmarElo - Live in São Paulo,Emicida: AmarElo - Live in São Paulo,3,Fred Ouro Preto,Fred Ouro Preto,1
1416,10.145862,0.999118,s241,s9048,The Witcher: Nightmare of the Wolf,The Witcher: Nightmare of the Wolf,3,Han Kwang Il,Han Kwang Il,1
1415,10.145862,0.999118,s146,s8953,House Party 2,House Party 2,3,"George Jackson, Doug McHenry","George Jackson, Doug McHenry",1
2908,10.145862,0.999118,s882,s9689,I Am All Girls,I Am All Girls,3,Donovan Marsh,Donovan Marsh,1
2907,10.145862,0.999118,s599,s9406,The American,The American,3,Anton Corbijn,Anton Corbijn,1
2316,10.145862,0.999118,s892,s9699,Layer Cake,Layer Cake,3,Matthew Vaughn,Matthew Vaughn,1
5509,10.145862,0.999118,s914,s9721,Time to Dance,Time to Dance,3,Stanley Menino D'Costa,Stanley Menino D'Costa,1


In [ ]:
from splink.diagnostics import splink_score_histogram
splink_score_histogram(df_e, spark, 100, "match_probability", symmetric=True)

In [ ]:
model.bayes_factor_chart()

df_e.coalesce(1).write.format("csv").option("header","true").mode("overwrite").save("/home/abhishekks446/data/output")

In [ ]:
model = linker.model
model.probability_distribution_chart()

In [ ]:
from splink.intuition import bayes_factor_intuition_chart
sample = df_e.toPandas().sample(1)
row_dict = sample.to_dict(orient="records")[0]
display(sample)
bayes_factor_intuition_chart(row_dict, model)

In [ ]:
cols = ['unique_id', 'director']
dfpd_l = df.toPandas().sample(300)[cols]
dfpd_l["join_col"] = 1
dfpd_r = dfpd_l.copy()
labels = dfpd_l.merge(dfpd_r, on = "join_col", suffixes=('_l', '_r'))
labels = labels[labels["unique_id_r"]< labels["unique_id_l"]]
labels["clerical_match_score"] = (labels["director_l"] == labels["director_r"]).astype(float)
labels = labels.drop(["director_l", "director_r", "join_col"], axis=1)
labels.head()

In [ ]:
from splink.truth import labels_with_splink_scores, roc_chart, precision_recall_chart
labels_sp = spark.createDataFrame(labels)
labels_and_scores = labels_with_splink_scores(labels_sp, df_e, "unique_id", spark)
labels_and_scores.show()

In [ ]:
labels_and_scores_pd = labels_and_scores.toPandas()
f1 = labels_and_scores_pd["clerical_match_score"] == 1.0
labels_and_scores_pd[f1].sample()

In [ ]:
from splink.cluster import clusters_at_thresholds

thresholds = {"cluster_0_5": 0.5, "cluster_0_9": 0.9}
clusters_at_thresholds(df, df_e, thresholds, linker.model).toPandas()